In [ ]:
#@foff
%profile profileName
%region ap-east-1
%iam_role arn:aws:iam::{number}:role/IamGlueJobName
%worker_type G.1X
%glue_version 4.0
%number_of_workers 10
%idle_timeout 20
%profile data-exchange
%session_id_prefix LUNA_SF_DT_HEALTH_CHECK
%connections LUNA-SNOWFLAKE-HEALTH-CHECK

In [ ]:
import sys
import json
import requests
import boto3
import functools

from pyspark.context import SparkContext

from awsglue.transforms import *
from awsglue.utils import getResolvedOptions
from awsglue.context import GlueContext
from awsglue.job import Job


sc = SparkContext.getOrCreate()
glueContext = GlueContext(sc)
spark = glueContext.spark_session

job = Job(glueContext)

default_arguments = {
    "--JOB_NAME": "snowflake-dynamic-table-health-check",
    "--DB_CONNECTION": "LUNA-SNOWFLAKE-HEALTH-CHECK",
    "--DATALAKE_REGION": "{Region}"
}

argv = [*sys.argv, *(a for pair in (kv for kv in default_arguments.items() if kv[0] not in sys.argv) for a in pair)]  
args = getResolvedOptions(argv, ['JOB_NAME', 'DB_CONNECTION', 'DATALAKE_REGION', 'IGNORE_TARGET1', 'IGNORE_TARGET2', 'IGNORE_TARGET3'])

job.init(args["JOB_NAME"], args)

class HealthCheck:
    def __init__(self):
        self._prod_slack_webhook_url = self._get_secret(env="prod")
        self._test_slack_webhook_url = self._get_secret(env="test")
    
    def __exception_handler(func):
        """
        수행될 각 Function 마다, Error 발생시 Slack 알람을 전달하기 위해 만든 decorator.
        Function에서 Error 발생시 _retry 함수로 Error가 발생한 Function을 전달 하여 처리한다. 
        :param func: Handler를 통해 전달 받을 Function.
        :return: wrapper -> function
        """
        @functools.wraps(func)
        def wrapper(self, *args, **kwargs):
            try:
                return func(self, *args, **kwargs)  
            except Exception as e:
                for env in ["prod", "test"]:
                    template = self._slack_body_template(env=env)
                    template.get('blocks')[1]['elements'][0]['elements'][0]['text'] = f'*Error Function_name*: {func.__name__}\n*Error Message*: {e}'
                    self._send_slack_message(env=env, slack_msg_template=template)
                return
        return wrapper 
    
    @__exception_handler
    def _get_secret(self, env: str):
        """
        AWS Secretmanager에서 Slack Webhook Url 정보를 가져 온다.
        각 환경에 맞는 Slack Webhook Url을 가져오도록 한다.
        :param env: String -> Secret Manager에서 가져올 Slack Channel의 env를 입력 한다.
        :return: str -> slack webhook url
        """
        secret_name = "prod/slack/webhook_url" if env == "prod" else "test/slack/webhook_url"
        channel = "notice-data_warehouse" if env == "prod" else "notice-data_warehouse-test"
        region_name = "{Region}"       
        # Create a Secrets Manager client
        session = boto3.session.Session()
        client = session.client(service_name='secretsmanager',region_name=region_name)
        get_secret_value_response = client.get_secret_value(SecretId=secret_name)
        secret_info = json.loads(get_secret_value_response["SecretString"])
        slack_webhook_url = secret_info[channel]
        return slack_webhook_url
    
    @__exception_handler
    def _query_exec(self, query: str):
        """
        Snowflake에 쿼리를 수행 하여 결과를 DynamicFrame으로 생성.
        :param query: String -> Snowflake로 수행할 쿼리 내용
        :return: DynamicFrame -> Snowflake Data
        """
        snowflake_data = glueContext.create_dynamic_frame.from_options(
        connection_type="snowflake",
        connection_options={
            "autopushdown": "off",
            "connectionName": "LUNA-SNOWFLAKE-HEALTH-CHECK",
            "sfDatabase": "DW_DX",
            "sfSchema": "PUBLIC",
            "sfWarehouse": "SVC_DX_WH",
            "query": query
            }
        )
        return snowflake_data
    
    @__exception_handler
    def _slack_body_template(self, env: str, failed_list: list = []):
        """
        Slack Message를 전달 할 Channel 환경을 입력 받아, 해당 채널로 Message를 전달하기 위한 Template 본문을 만든다.
        전달할 failed_list가 존재할 경우 block 본문에 section을 추가하여 전달 한다.
        failed_list가 없을 경우엔, "There are No Failed Lists." 문구를 추가하여 알람 전달.
        :param env: Slack Message를 전달할 환경을 기입한다.
        :param failed_list: default value는 빈 list. Slack Block Body를 만들 Message가 담긴 Failed Data List.
        :return: dict
        """
        slack_message_template = {
            "channel": "notice-data_warehouse" if env == "prod" else "notice-data_warehouse-test",
            "username": "Snowflake",
            "blocks": [
                {
                    "type": "header",
                    "text": {
                        "type": "plain_text",
                        "text": "Snowflake DT Health Check",
                        "emoji": True,
                    },
                }
            ]
        }
        if failed_list:
            for row in failed_list:       
                slack_message_template.get("blocks").append(
                    {
                        "type": "section",
                        "text": {
                            "type": "mrkdwn",
                            "text": f"Failed:\n*{row[0]}*\n▪ reason_code: {row[1]['reason_code']}\n▪ reason_message: {row[1]['reason_message']}\n▪ state: {row[1]['state']}\n▪ suspended_on: {row[1]['suspended_on']}"
                            }
                    }
                )
        else:
            slack_message_template.get("blocks").append(
                {
                    "type": "rich_text",
                    "elements": [
                        {
                            "type": "rich_text_section",
                            "elements": [
                                {
                                    "type": "text",
                                    "text": "There are No Failed Lists.",
                                }
                            ],
                        }
                    ]
                }
            )
        return slack_message_template
        
    @__exception_handler
    def _send_slack_message(self, env: str, slack_msg_template: dict = None):
        """
        Slack으로 메세지를 전송.
        status_code가 200인 경우 성공 그렇지 않은 경우 메세지 전송에 싪패했다는 문구를 표출 한다. 
        :param env: String -> Message를 전달할 Slack Channel의 env를 입력 한다.
        :return: None or String(Slack 메세지 전송에 실패할 경우 실패 문구를 담은 String 변수를 반환.)
        """
        response = requests.post(url=self._prod_slack_webhook_url if env == "prod" else self._test_slack_webhook_url, data=json.dumps(slack_msg_template), headers={'Content-Type': 'application/json'})
        if response.status_code == 200:
            print(f"{env} 환경 Message 전송 Success")
            return None
        else:
            print(f"{env} 환경 Message 전송 Failed")
            failed_msg = f"{env} 환경 Slack Message 전송 Failed"
            return failed_msg
    
    @__exception_handler
    def _ignore_dt(self, failed_list: list):
        """
        Dynamic Table Health Check시 Ignore 할 Target들을 제외한, 새로운 List를 생성 하여 반환 한다. 
        - Ignore Type(Glue Parameter 값 입력 규칙) 
            1. 특정 DB만 Ignore 하고 싶은 경우 
                - Sample
                    --IGNORE_TARGET1: ["DW_DX"] 
            2. 특정 DB의 Schema 만 Ignore 하고 싶은 경우
                - Sample
                    --IGNORE_TARGET2: ["DW_DX.LUNA_TEST"]
            3. 특정 Table 1건만 Ignore 하고 싶은 경우 
                - Sample
                    --IGNORE_TARGET3: ["DW_DX.LUNA_TEST.DT_TEST_1"]
        - 함수 동작 로직
            1. DT Health Check Failed List 를 for 문을 통해 한 건씩 DT Name을 확인 한다.
            2. Failed List 의 각각 한 건마다 Ignore List 에 해당 되는 대상이 존재 하는지 확인 한다. 
                - Ignore Target 에 해당 DT의 DB 이름이 존재할 경우 flag 값을 True 로 변경 하고 더 이상의 Ignore 조건을 검색 하지 않도록 Inner for 문에서 빠져 나간다. 
                  그리고 첫번째 for 문 으로 돌아 가서 다음 Failed_list 확인 작업을 이어서 수행 한다.
                - Ignore Target 의 첫번째 조건인 DB 명에서 제외 되지 않을 경우 elif 문으로 넘어 가서 DB 이름 + Schema 이름을 확인 하여 Ignore 대상에 해당 되는지 확인 한다. 
                - 위 Schema 명에서 까지 제외 되지 않을 경우 DB 이름 + Schema 이름 + Table 이름을 확인 하여 Ignore 대상에 해당 되는지 확인 한다.
                - 위 세가지 조건을 모두 확인한 결과, 어떤 조건 에도 해당 되지 않을 경우 result_list 에 추가 하여 새로운 List 를 만든다.
            3. 새롭게 생성한 result_list 를 반환 한다.
        - Data Sample
            - failed_list: [['DW_DX.LUNA_TEST.GGCORE_AGENT', {'reason_code': 'USER_SUSPENDED', 'reason_message': 'The DT was suspended by a user', 'state': 'SUSPENDED', 'suspended_on': 'Z 2024-05-21 01:46:35.796'}], [...]]
            - dt_path: ["DW_DX", "LUNA_TEST", "TEST_DT_1"]
            - parse_ignore_list: [['DW_DX', 'DW_DX.LUNA_TEST.TEST_DT_1'], ['DW_TEST.LUNA_TEST', 'DW_WAREHOUSE', 'DW_DX.PUBLIC'], [...]] or [[],[],[]]
            
        :return: list -> DT Health Check Failed List Except Ignore List.
        """
        result_list = list()
        
        ignore_list = [args[f"IGNORE_TARGET{i}"] for i in range(1, 4)]
        parse_ignore_list = [json.loads(item) for item in ignore_list]
         
        for item in failed_list: 
            dt_path = item[0].split(".") 
            flag = False
            for ignore_target in parse_ignore_list:
                if dt_path[0] in ignore_target:
                    flag = True
                    break
                elif ".".join(dt_path[:2]) in ignore_target:
                    flag = True
                    break
                elif ".".join(dt_path) in ignore_target:
                    flag = True
                    break
            if not flag:  
                result_list.append(item)
        return result_list    
    
    @__exception_handler
    def _failed_list_separate(self, final_failed_list: list):
        """
        환경별 Slack Channel로 분기하여 알람을 전달하기 위해,
        최종적으로 Failed DT List를 환경 별로 분리하는 작업을 수행 한다.
        :param final_failed_list: 최종적으로 Slack 알람으로 전달될 Faile DT List -> [[]. [], ...] or []
            - sample: [['DW_COMPLIANCE_TEST.PUBLIC.PLAYER_CLOSE_REPORT', {'reason_code': 'SUSPENDED_DUE_TO_ERRORS', 'reason_message': 'The DT was suspended due to 5                          consecutive refresh errors', 'state': 'SUSPENDED', 'suspended_on': 'Z 2024-11-20 14:01:12.769'}], 
                       ['DW_COMPLIANCE_TEST.PUBLIC.PLAYER_ADJUSTMENT_REPORT', {'reason_code': 'SUSPENDED_DUE_TO_ERRORS', 'reason_message': 'The DT was suspended due to 5 consecutive refresh errors', 'state': 'SUSPENDED', 'suspended_on': 'Z 2024-11-20 14:01:12.249'}]]
        :return: prod_list, test_list -> 각각 Prod Slack channel로 전달될 list와 Test channel로 전달될 list를 의미함.
        """
        prod_list, test_list  = list(), list()
        
        for item in final_failed_list:
            env = item[0].split(".")[0].split("_")
            if "TEST" in env or "STAGE" in env:
                test_list.append(item)
            else:
                prod_list.append(item)
        return prod_list, test_list
    
    @__exception_handler
    def execute(self):
        """
        Main으로 실행될 함수.
        1. Snowflake에 Query를 수행하여 원하는 결과를 가져와서 DynamicFrame으로 생성.
        2. DynamicFrame을 DataFrame.collect 객체로 변환.
        3. DataFrame을 for문으로 반복하여 state = "SUSPENDED" and reason_code not in ("USER_SUSPENDED","UPSTREAM_USER_SUSPENDED")인 것들만 Failed_list에 담는다. 
        4. Failed_list에서 Health Check 알람 전달시 Ignore할 List를 제외한 최종 결과물인 final_failed_list를 만들어 낸다.  
        5. final_failed_list를 환경에 맞는 Slack Channel로 알람을 전송 하기 위해 분류 하는 작업을 한다.
        5. 최종적으로 Prod, Test Slack Channel로 알람이 전송된다. 
            - 실패한 DT List 가 존재할 경우 실패한 대상이 전송 되고, 실패한 대상이 없을 경우 "There are No Failed Lists." 문구가 전달 된다.
        :return: None 
        """        
        dyf_query_result = self._query_exec(query="SELECT NAME, QUALIFIED_NAME, SCHEDULING_STATE FROM TABLE (INFORMATION_SCHEMA.DYNAMIC_TABLE_GRAPH_HISTORY())") # DynamicFrame
        data_list = dyf_query_result.toDF().collect()

        failed_list = []

        for row in data_list:
            json_data = json.loads(row["SCHEDULING_STATE"])
            if json_data.get("state") == "SUSPENDED" and json_data.get("reason_code") not in ["USER_SUSPENDED","UPSTREAM_USER_SUSPENDED"]:   
                failed_list.append([row["QUALIFIED_NAME"], json_data])
        ## Ignore 대상 제외 후, Slack으로 전달할 최종 Failed List.
        final_failed_list = self._ignore_dt(failed_list=failed_list)
        ## Slack Channel 환경에 맞도록 분리하여 Message 전달.
        final_prod_list, final_test_list = self._failed_list_separate(final_failed_list = final_failed_list)
        for env, failed_list in ['prod', final_prod_list], ['test', final_test_list]:
            slack_msg_template = self._slack_body_template(env=env, failed_list=failed_list)
            self._send_slack_message(env=env, slack_msg_template=slack_msg_template)

hc = HealthCheck()
hc.execute()